In [1]:
import numpy as np
from tqdm import tqdm
from scipy.special import softmax
from onnxruntime import GraphOptimizationLevel, InferenceSession, SessionOptions, get_all_providers
from gpt2_tokenizer import GPT2Tokenizer

In [2]:
def create_model_for_provider(model_path: str, provider: str= 'CPUExecutionProvider') -> InferenceSession:
    assert provider in get_all_providers(), f"provider {provider} not found, {get_all_providers()}"
    # Few properties that might have an impact on performances (provided by MS)
    options = SessionOptions()
    options.intra_op_num_threads = 4
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    # Load the model as a graph and prepare the CPU backend
    session = InferenceSession(model_path, options, providers=[provider])
    session.disable_fallback()
    return session

In [3]:
tokenizer = GPT2Tokenizer(
    'CPM-Generate/bpe_3w_new/vocab.json',
    'CPM-Generate/bpe_3w_new/merges.txt',
    model_file='CPM-Generate/bpe_3w_new/chinese_vocab.model')

In [4]:
cpm = create_model_for_provider('./onnx_q/cpm.onnx')

In [5]:
cpm_kv = create_model_for_provider('./onnx_kv_q/cpm.onnx')

In [6]:
print('done')

done


In [42]:
%%time
ids = tokenizer.encode('你好')

input_ids = np.array([ids], dtype=np.int64)

logits, new_kv = cpm.run(None, {
    "input_ids": input_ids,
})
kv_cache = new_kv

next_token = logits[0, -1, :].argmax()

outputs = [next_token]
for i in range(200):
    input_ids = np.array([[next_token]], dtype=np.int64)
    logits, new_kv = cpm_kv.run(None, {
        "input_ids": np.array([[next_token]], dtype=np.int64),
        'kv_cache': kv_cache,
    })
    next_token = logits[0, -1, :].argmax()
    kv_cache = np.concatenate([kv_cache, new_kv], axis=-2)
    # kv_cache = kv_cache[:, :, :, :, -50:, :]
    outputs.append(next_token)
print(len(outputs), tokenizer.decode(outputs))

201 , 我 是 个 很 好 的 人 , 我 也 很 喜欢 你 , 但是 我 不想 和 你 在 一起 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我 想 和 你 做 朋友 , 我
CPU times: user 1min 32s, sys: 4.38 s, total: 1min 36s
Wall time: 25.3 s


In [44]:
# %%time
# ids = tokenizer.encode('你好')

# input_ids = np.array([ids], dtype=np.int64)

# logits, new_kv = cpm.run(None, {
#     "input_ids": input_ids,
# })
# kv_cache = new_kv

# next_token = logits[0, -1, :].argmax()

# outputs = [next_token]
# for i in range(200):
#     input_ids = np.array([[next_token]], dtype=np.int64)
#     logits, new_kv = cpm_kv.run(None, {
#         "input_ids": np.array([[next_token]], dtype=np.int64),
#         'kv_cache': kv_cache,
#     })
#     next_token = logits[0, -1, :].argmax()
#     kv_cache = np.concatenate([kv_cache, new_kv], axis=-2)
#     kv_cache = kv_cache[:, :, :, :, -50:, :]
#     outputs.append(next_token)
# print(len(outputs), tokenizer.decode(outputs))

In [35]:
# %%time
# ids = tokenizer.encode('你好')

# outputs = []
# for i in range(200):
#     input_ids = np.array([ids], dtype=np.int64)
#     logits = cpm.run([cpm.get_outputs()[0].name], {
#         "input_ids": input_ids,
#     })[0]
#     next_token = logits[0, -1, :].argmax()
#     ids.append(next_token)
#     outputs.append(next_token)
# print(len(outputs), tokenizer.decode(outputs))

In [55]:
# ids = tokenizer.encode('上联：天下太平\n下联：')

def generate(
    text,
    max_len = 100,
    temperature = 1.0,
    top_p = 0.95,
    top_k = 50,
    eod=tokenizer.eod_id,
    ban = [
        8,  # 一个空白字符
    ]):

    ids = tokenizer.encode(text)

    kv_cache = None

    for i in range(max_len):

        if i == 0:
            logits, kv_cache = cpm.run(None, {
                "input_ids": np.array([ids], dtype=np.int64)
            })
        else:
            logits, new_kv = cpm_kv.run(None, {
                "input_ids": np.array([[next_token]], dtype=np.int64),
                'kv_cache': kv_cache,
            })
            kv_cache = np.concatenate([kv_cache, new_kv], axis=-2)

        for x in ban:
            logits[:, -1, x] = -9999

        logits = logits / temperature
        scores = softmax(logits[:, -1, :])
        next_probs = np.sort(scores)[:, ::-1]
        if top_p > 0.0 and top_p < 1.0:
            next_probs = next_probs[:, :int(next_probs.shape[1] * (1 - top_p))]
        if top_k > 0 and top_k < next_probs.shape[1]:
            next_probs = next_probs[:, :top_k]
        next_probs_1 = next_probs / next_probs.sum(axis=1).reshape((-1, 1))

        next_tokens = np.argsort(scores)[:, ::-1]
        if top_p > 0.0 and top_p < 1.0:
            next_tokens = next_tokens[:, :int(next_tokens.shape[1] * (1 - top_p))]
        if top_k > 0 and top_k < next_tokens.shape[1]:
            next_tokens = next_tokens[:, :top_k]

        next_token = np.random.choice(next_tokens[0], p=next_probs_1[0])
        if eod is not None:
            if eod == next_token:
                break
        ids.append(next_token)
    return tokenizer.decode(ids).replace(' ', '')

In [57]:
generate('天下是否太平，取决于')

'天下是否太平,取决于他们的个人能力与责任感的高低!'

In [61]:
generate('我是一个孤寡老人，我')

'我是一个孤寡老人,我的儿子和孙子都不在我身边了!我知道他们很想念我!我知道他们很爱我!我也只有老伴了!不知道我还有其他没出世的孩子吗?都多大啦?什么时候来接我一下呀?等着我给你们带礼物呢?'